```{contents}
```

# Memory Management

## Contiguous memory blocks

* NumPy arrays (`ndarray`) store elements in a **single contiguous block** of memory.
* All elements have the same fixed size (`dtype`).
* This allows fast vectorized operations using CPU instructions (SIMD).

---

## Array metadata

Each `ndarray` has two parts:

1. **Data buffer** → raw memory with values.
2. **Metadata** → tells NumPy how to interpret the buffer:

   * `shape`: dimensions
   * `dtype`: element type
   * `strides`: step (in bytes) to move along each axis
   * `order`: memory layout (C or Fortran)

The metadata is lightweight compared to the data buffer.

---

## Strides and views

* Views reuse the same memory with different `strides` and `shape`.
* Example: transpose, slicing.
* No copy happens, only metadata changes.



Here is a **focused code walkthrough** on **strides and views** in NumPy:

---

## Basic strides


In [7]:
import numpy as np

a = np.arange(12).reshape(3,4)   # 3x4 array
print("Array:\n", a)
print("Shape:", a.shape)
print("Strides:", a.strides)


Array:
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
Shape: (3, 4)
Strides: (32, 8)



* `dtype=int64` → 8 bytes per element
* Strides `(32, 8)` →

  * Row stride = 4 × 8 = 32 bytes
  * Column stride = 8 bytes

---

## Transpose (changes strides, not data)


In [8]:
b = a.T
print("\nTranspose:\n", b)
print("Shape:", b.shape)
print("Strides:", b.strides)
print("Shares memory:", b.base is a)



Transpose:
 [[ 0  4  8]
 [ 1  5  9]
 [ 2  6 10]
 [ 3  7 11]]
Shape: (4, 3)
Strides: (8, 32)
Shares memory: False




* Strides swapped → `(8, 32)`
* No copy, just different view.

---

## Slicing (skipping columns)


In [9]:
c = a[:, ::2]   # every 2nd column
print("\nSlice every 2nd column:\n", c)
print("Shape:", c.shape)
print("Strides:", c.strides)
print("Shares memory:", c.base is a)


Slice every 2nd column:
 [[ 0  2]
 [ 4  6]
 [ 8 10]]
Shape: (3, 2)
Strides: (32, 16)
Shares memory: False


* Column stride doubles: `(32, 16)`
* Still a view on `a`.

---

## Forcing a copy

In [ ]:
d = a[:, ::2].copy()
print("\nCopy of slice:\n", d)
print("Shares memory:", d.base is a)


Copy of slice:
 [[ 0  2]
 [ 4  6]
 [ 8 10]]
Shares memory: False




* Copy breaks the link → new memory.

---

## Broadcasting creates fake strides


In [11]:
e = np.array([1,2,3])
f = np.ones((3,3), dtype=int)
g = e + f
print("\nBroadcast result:\n", g)
print("e.strides:", e.strides)
print("Broadcasted shape:", g.shape)



Broadcast result:
 [[2 3 4]
 [2 3 4]
 [2 3 4]]
e.strides: (8,)
Broadcasted shape: (3, 3)



* Broadcasting simulates repetition using stride tricks instead of copying.

**Key takeaway**:

* **Views** = same data buffer, different strides.
* **Copies** = new memory allocation.
* Strides are the mechanism that lets NumPy slice, transpose, and broadcast **without duplicating data**.




```python
import numpy as np
a = np.arange(12).reshape(3,4)
b = a.T
print(b.base is a)  # True → shares memory
```


## Copies vs views

* **View**: No extra memory, just different strides.
* **Copy**: New memory allocation. Happens when:

  * Changing dtype
  * Forcing contiguity (`np.ascontiguousarray`)
  * Some reshapes (e.g., `.ravel(order='F')` on C-ordered array).


In [12]:
import numpy as np

a = np.arange(10)
b = a[2:7]   # slice
b[0] = 99    # modify slice
print("Original:", a)   # a is changed
print("Slice:", b)
print("Is view:", b.base is a)


Original: [ 0  1 99  3  4  5  6  7  8  9]
Slice: [99  3  4  5  6]
Is view: True


In [14]:
c = a[2:7].copy()
c[0] = -1
print("Original after copy change:", a)  # unchanged
print("Copy:", c)
print("Is copy:", c.base is a)

Original after copy change: [ 0  1 99  3  4  5  6  7  8  9]
Copy: [-1  3  4  5  6]
Is copy: False



---

## Memory order

* **C-order (row-major)**: last axis changes fastest (default in NumPy).
* **Fortran-order (column-major)**: first axis changes fastest.
* Impacts performance in loops and external library compatibility.



In [15]:
import numpy as np

a = np.array([[1,2,3],
              [4,5,6]], order='C')

print("Array (C-order):\n", a)
print("Shape:", a.shape)
print("Strides:", a.strides)
print("Flatten (C-order):", a.ravel(order='C'))
print("Flatten (F-order):", a.ravel(order='F'))


Array (C-order):
 [[1 2 3]
 [4 5 6]]
Shape: (2, 3)
Strides: (24, 8)
Flatten (C-order): [1 2 3 4 5 6]
Flatten (F-order): [1 4 2 5 3 6]


In [17]:
b = np.array([[1,2,3],
              [4,5,6]], order='F')

print("\nArray (F-order):\n", b)
print("Shape:", b.shape)
print("Strides:", b.strides)
print("Flatten (C-order):", b.ravel(order='C'))
print("Flatten (F-order):", b.ravel(order='F'))



Array (F-order):
 [[1 2 3]
 [4 5 6]]
Shape: (2, 3)
Strides: (8, 16)
Flatten (C-order): [1 2 3 4 5 6]
Flatten (F-order): [1 4 2 5 3 6]


In [18]:
print("a.flags.c_contiguous:", a.flags.c_contiguous)
print("a.flags.f_contiguous:", a.flags.f_contiguous)

print("b.flags.c_contiguous:", b.flags.c_contiguous)
print("b.flags.f_contiguous:", b.flags.f_contiguous)


a.flags.c_contiguous: True
a.flags.f_contiguous: False
b.flags.c_contiguous: False
b.flags.f_contiguous: True




---

## Broadcasting

* NumPy does not copy smaller arrays when shapes differ.
* It simulates repetition by adjusting **strides** (zero strides for broadcasted dimensions).
* Saves memory, enables vectorized operations.

---

## Underlying implementation

* NumPy arrays are thin wrappers around **C-contiguous buffers**.
* Uses `malloc` / `free` internally, not Python’s object memory model.
* This avoids Python’s object overhead for each element.

---

## Memory efficiency

* Fixed-size homogeneous elements → compact storage.
* Much smaller than Python lists (which store full Python objects + pointers).
* Example:

  * `list` of 1e6 ints → \~35–40 MB
  * `np.array` of 1e6 int32 → \~4 MB

---

**Summary**
NumPy manages memory by:

* Storing data in contiguous blocks
* Using fixed dtypes
* Exposing memory via `shape`, `strides`, and `order`
* Returning views when possible to avoid copies
* Broadcasting with stride tricks instead of replication

